In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import os, sys

# 打开文件
path = "./MRT_raw_data"
dirs = os.listdir( path )
adjacent = pd.read_csv('MRT_raw_data/相鄰站數.csv')

,日期,累積確診人數,確診人數,經濟防疫新模式,二級警戒,三級警戒,是否7+7,是否3+4,首例確診,口罩實名制,簡訊實聯制上路,Delta變異株,Omicron亞型變異株
0,2017-01-01,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-01-02,0,0,0,0,0,0,0,0,0,0,0,0
2,2017-01-03,0,0,0,0,0,0,0,0,0,0,0,0
3,2017-01-04,0,0,0,0,0,0,0,0,0,0,0,0
4,2017-01-05,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2033,2022-07-27,4520777,23934,1,0,0,0,1,1,0,0,1,1
2034,2022-07-28,4544017,23240,1,0,0,0,1,1,0,0,1,1
2035,2022-07-29,4565479,21462,1,0,0,0,1,1,0,0,1,1
2036,2022-07-30,4586523,21044,1,0,0,0,1,1,0,0,1,1


In [9]:
# ['年','月份','站名','進站','出站']
for file in dirs : 
    df = pd.read_csv(f"MRT_raw_data/{file}")
        
    influx = df.groupby(['日期','進站','時段']).sum() #進站人次
    outflux = df.groupby(['日期','出站','時段']).sum() #出站人次
    
    influx_index = np.array([*influx.index]) # https://blog.csdn.net/weixin_42423817/article/details/118793657?spm=1001.2101.3001.6650.13&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&utm_relevant_index=19
    influx_index = pd.DataFrame(influx_index) # 把 index 分離出來 變成一欄 dataframe
    
    in_cnt=pd.DataFrame(influx.values) # 把 in 的 value 分離出來 變成 dataframe
    out_cnt=pd.DataFrame(outflux.values) # 把 out 的 value 分離出來 變成 dataframe
    
    cooked = pd.concat([influx_index, in_cnt, out_cnt] , axis=1) # 合併 index, in的value, out的value
    cooked.columns = ['日期','站名','時段','進站','出站'] # 設定欄位名稱
    
    
    cooked['月份'] = cooked['日期'].str[5:7] # 新增月份欄位
    cooked['年份'] = cooked['日期'].str[:4] # 新增年份欄位
    
    cooked = cooked[['日期','年份','月份','站名','時段','進站','出站']] # 調整欄位順序(日期相關的顯示在前面)
    #print(f)
    df_each_station = cooked.groupby(['年份','月份','站名']).sum() # 只需要每月每站總流量
    df_each_station = df_each_station[["進站", "出站"]]
    
    influx_index = np.array([*df_each_station.index])
    influx_index = pd.DataFrame(influx_index)
    
    in_cnt = pd.DataFrame(df_each_station['進站'].values)
    out_cnt = pd.DataFrame(df_each_station['出站'].values)
    
    cooked_each_station = pd.concat([influx_index, in_cnt, out_cnt] , axis=1)
    cooked_each_station.columns = ['年','月份','站名','進站','出站']
    cooked_each_station = pd.merge(cooked_each_station,adjacent, how='inner', on=['站名']) # 合併相鄰站數
    print(file)
    if not os.path.isdir('MRT_cooked_data'):
        os.mkdir('MRT_cooked_data')
    cooked_each_station.to_csv(f"MRT_cooked_data/cooked_MRT_by_Month_{file}", index=False)
    

201701.csv
201702.csv
201703.csv
201704.csv
201705.csv
201706.csv
201707.csv
201708.csv
201709.csv
201710.csv
201711.csv
201712.csv
201801.csv
201802.csv
201803.csv
201804.csv
201805.csv
201806.csv
201807.csv
201808.csv
201809.csv
201810.csv
201811.csv
201812.csv
201901.csv
201902.csv
201903.csv
201904.csv
201905.csv
201906.csv
201907.csv
201908.csv
201909.csv
201910.csv
201911.csv
201912.csv
202001.csv
202002.csv
202003.csv
202004.csv
202005.csv
202006.csv
202007.csv
202008.csv
202009.csv
202010.csv
202011.csv
202012.csv
202101.csv
202102.csv
202103.csv
202104.csv
202105.csv
202106.csv
202107.csv
202108.csv
202109.csv
202110.csv
202111.csv
202112.csv
202201.csv
202202.csv
202203.csv
202204.csv
202205.csv
202206.csv
202207.csv


KeyError: '日期'

In [19]:
df_merged = pd.read_csv(f"MRT_cooked_data/cooked_MRT_by_Month_202205.csv")
df_merged.head(5)

,年,月份,站名,進站,出站,相鄰站數
0,2022,5,BL板橋,566676,628540,3
1,2022,5,Y板橋,142094,140027,3
2,2022,5,七張,256576,260349,3
3,2022,5,三和國中,204831,199065,2
4,2022,5,三民高中,193853,198328,2


In [6]:
    df = pd.read_csv(f"MRT_raw_data/202204.csv")
        
    influx = df.groupby(['日期','進站','時段']).sum() #進站人次
    outflux = df.groupby(['日期','出站','時段']).sum() #出站人次
    
    influx_index = np.array([*influx.index]) # https://blog.csdn.net/weixin_42423817/article/details/118793657?spm=1001.2101.3001.6650.13&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&utm_relevant_index=19
    influx_index = pd.DataFrame(influx_index) # 把 index 分離出來 變成一欄 dataframe
    
    in_cnt=pd.DataFrame(influx.values) # 把 in 的 value 分離出來 變成 dataframe
    out_cnt=pd.DataFrame(outflux.values) # 把 out 的 value 分離出來 變成 dataframe
    
    cooked = pd.concat([influx_index, in_cnt, out_cnt] , axis=1) # 合併 index, in的value, out的value
    cooked.columns = ['日期','站名','時段','進站','出站'] # 設定欄位名稱
    
    
    cooked['月份'] = cooked['日期'].str[5:7] # 新增月份欄位
    cooked['年份'] = cooked['日期'].str[:4] # 新增年份欄位
    
    cooked = cooked[['日期','年份','月份','站名','時段','進站','出站']] # 調整欄位順序(日期相關的顯示在前面)
    #print(f)
    df_each_station = cooked.groupby(['年','月份','站名']).sum()
    df_each_station = df_each_station[["進站", "出站"]]
    
    influx_index = np.array([*df_each_station.index])
    influx_index = pd.DataFrame(influx_index)
    
    in_cnt = pd.DataFrame(df_each_station['進站'].values)
    out_cnt = pd.DataFrame(df_each_station['出站'].values)
    
    cooked_each_station = pd.concat([influx_index, in_cnt, out_cnt] , axis=1)
    cooked_each_station.columns = ['日期','年','月份','站名','進站','出站']
    cooked_each_station = pd.merge(cooked_each_station,adjacent, how='inner', on=['站名']) # 合併相鄰站數
    
    if not os.path.isdir('MRT_cooked_data'):
        os.mkdir('MRT_cooked_data')
    cooked_each_station.to_csv(f"MRT_cooked_data/cooked_MRT_by_Month_202204.csv", index=False)

In [ ]:
path2 = "./MRT_cooked_data"
dirs2 = os.listdir( path )
for file in dirs2:
   print (file)

In [12]:
df = pd.read_csv(f"MRT_cooked_data/cooked_MRT_by_Month_202205.csv")
pop_df = pd.read_csv(f"POP_cooked_data/cooked_202205.csv")
neighbor = pd.read_csv(f"MRT_neighboring/MRT_neighboring.csv")
merged = pd.merge(df, pop_df, how='inner', on=['站名'])
merged = pd.merge(merged,neighbor,how='inner', on=['站名'])
merged

,年,月份,站名,進站,出站,相鄰站數,人口數,男,女,公司數,學校數,鄰近股份有限公司數量,鄰近學校數量,鄰近醫療數量,鄰近嫌避數量
0,2022,5,BL板橋,566676,628540,3,126358.0,61920.0,64438.0,160.0,10.0,299,39,4,1
1,2022,5,Y板橋,142094,140027,3,50258.0,24095.0,26163.0,125.0,4.0,167,8,0,1
2,2022,5,七張,256576,260349,3,50465.0,23764.0,26701.0,47.0,7.0,267,47,3,4
3,2022,5,三和國中,204831,199065,2,27355.0,13288.0,14067.0,32.0,3.0,172,35,0,5
4,2022,5,三民高中,193853,198328,2,63320.0,31016.0,32304.0,35.0,4.0,110,50,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2022,5,頂埔,154636,147439,1,27328.0,13508.0,13820.0,183.0,2.0,263,5,1,2
115,2022,5,頂溪,573134,593636,2,128545.0,61062.0,67483.0,129.0,9.0,197,30,2,0
116,2022,5,頭前庄,102044,100776,4,41377.0,20051.0,21326.0,62.0,2.0,187,3,0,1
117,2022,5,麟光,76487,71653,2,11842.0,5601.0,6241.0,8.0,0.0,75,0,0,0


In [15]:
months =  ['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205','202206', '202207']
for m in months:
    df = pd.read_csv(f"MRT_cooked_data/cooked_MRT_by_Month_{m}.csv")
    pop_df = pd.read_csv(f"POP_cooked_data/cooked_{m}.csv")
    neighbor = pd.read_csv(f"MRT_neighboring/MRT_neighboring.csv")
    
    merged = pd.merge(df, pop_df, how='inner', on=['站名'])
    merged = pd.merge(merged,neighbor,how='inner', on=['站名'])
    if not os.path.isdir('Merged_cooked_data'):
        os.mkdir('Merged_cooked_data')
    merged.to_csv(f"Merged_cooked_data/Merged_{m}.csv", index=False)
    

In [18]:
files = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207']
df_all = pd.read_csv(f"Merged_cooked_data/Merged_201701.csv")
for f in files[1:]:
    df = pd.read_csv(f"Merged_cooked_data/Merged_{f}.csv")
    df_all = pd.concat([df_all,df], ignore_index=True)
    if not os.path.isdir('Merged_data'):
        os.mkdir('Merged_data')
    df_all.to_csv(f"Merged_data/Merged_all.csv", index=False)
    
    print(f,"stacked")
print(df_all.shape[0])

201702 stacked
201703 stacked
201704 stacked
201705 stacked
201706 stacked
201707 stacked
201708 stacked
201709 stacked
201710 stacked
201711 stacked
201712 stacked
201801 stacked
201802 stacked
201803 stacked
201804 stacked
201805 stacked
201806 stacked
201807 stacked
201808 stacked
201809 stacked
201810 stacked
201811 stacked
201812 stacked
201901 stacked
201902 stacked
201903 stacked
201904 stacked
201905 stacked
201906 stacked
201907 stacked
201908 stacked
201909 stacked
201910 stacked
201911 stacked
201912 stacked
202001 stacked
202002 stacked
202003 stacked
202004 stacked
202005 stacked
202006 stacked
202007 stacked
202008 stacked
202009 stacked
202010 stacked
202011 stacked
202012 stacked
202101 stacked
202102 stacked
202103 stacked
202104 stacked
202105 stacked
202106 stacked
202107 stacked
202108 stacked
202109 stacked
202110 stacked
202111 stacked
202112 stacked
202201 stacked
202202 stacked
202203 stacked
202204 stacked
202205 stacked
202206 stacked
202207 stacked
8139


In [ ]:
# Reference
for f in files:
    df = pd.read_csv(f"MRT_raw_data/{f}.csv")
        
    influx = df.groupby(['日期','進站','時段']).sum() #進站人次
    outflux = df.groupby(['日期','出站','時段']).sum() #出站人次
    
    influx_index = np.array([*influx.index]) # https://blog.csdn.net/weixin_42423817/article/details/118793657?spm=1001.2101.3001.6650.13&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7EBlogCommendFromBaidu%7Edefault-13-118793657-blog-80330077.pc_relevant_multi_platform_whitelistv2&utm_relevant_index=19
    influx_index = pd.DataFrame(influx_index) # 把 index 分離出來 變成一欄 dataframe
    
    in_cnt=pd.DataFrame(influx.values) # 把 in 的 value 分離出來 變成 dataframe
    out_cnt=pd.DataFrame(outflux.values) # 把 out 的 value 分離出來 變成 dataframe
    
    cooked = pd.concat([influx_index, in_cnt, out_cnt] , axis=1) # 合併 index, in的value, out的value
    cooked.columns = ['日期','站名','時段','進站','出站'] # 設定欄位名稱
    
    # cooked = pd.merge(cooked,stn_area) # 合併行政區(後來改用里, 故此行註解掉)
    
    cooked['月份'] = cooked['日期'].str[5:7] # 新增月份欄位
    cooked['年份'] = cooked['日期'].str[:4] # 新增月份欄位
    cooked['星期'] = pd.to_datetime(cooked['日期']).dt.dayofweek # 新增星期欄位
    cooked['週末'] = (cooked['星期'] // 5 == 1) # 六,日為5,6 新增周末欄位
    
    cooked = cooked[['日期','年份','月份','星期','週末','站名','時段','進站','出站']] # 調整欄位順序(日期相關的顯示在前面)

    cooked = pd.merge(cooked,adjacent, how='inner', on=['站名']) # 合併相鄰站數
    
    print(f)
    
    if not os.path.isdir('MRT_cooked_data'):
        os.mkdir('MRT_cooked_data')
    cooked.to_csv(f"MRT_cooked_data/cooked_{f}.csv", index=False)
    